In [1]:
using LinearAlgebra, Plots, Random, Distributions

First, let us create a NamedTuple with our parameters.

In [20]:
params = (; β = 0.72, # bargaining_power
            r = 0.012, # discount rate
            λ = 4, # arrival rate
            z = 0.4, # value of leisure
            p_star = 1., # productivity
            s_star = 0.1, # separation rate
            c = 0.213, # cost of vacancy
            σ = 0.0165, # standard deviation 
            γ = 0.004, # autoregressive coefficient
            n = 2001, # gridpoints
            lb = -1., # bounds for grid
            ub = 1., # bounds for grid
            mc = 1.355, # constant term in matching function
            me = -0.72 ) # exponent in matching function

(β = 0.72, r = 0.012, λ = 4, z = 0.4, p_star = 1.0, s_star = 0.1, c = 0.213, σ = 0.0165, γ = 0.004, n = 2001, lb = -1.0, ub = 1.0, mc = 1.355, me = -0.72)

In [21]:
function matching_function(grid, params)
    (; mc, me) = params
    return mc .* grid .^ me
end

matching_function (generic function with 1 method)

In [22]:
function generate_income_process(params)
    (; n, lb, ub) = params
    Δ = ub / n
    grid = LinRange(lb, ub, n)
    exp_grid = exp.(grid)
    transition_matrix = zeros(n, n)
    for i in 1:n
        for j in 1:n
            if i == 1
                if j == 1
                    transition_matrix[i, j] = ( 1/2 ) * (1 .+ (grid[i] / (n*Δ)))
                elseif j == 2
                    transition_matrix[i, j] = ( 1/2 ) * (1 .- (grid[i] / (n*Δ)))
                end
            elseif i == n
                if j == n-1
                    transition_matrix[i, j] = ( 1/2 ) * (1 .+ (grid[i] / (n*Δ)))
                elseif j == n
                    transition_matrix[i, j] = ( 1/2 ) * (1 .- (grid[i] / (n*Δ)))
                end
            else
                if j == i - 1
                    transition_matrix[i, j] = ( 1/2 ) * (1 .+ (grid[i] / (n*Δ)))
                elseif j == i + 1
                    transition_matrix[i, j] = ( 1/2 ) * (1 .- (grid[i] / (n*Δ)))
                end
            end
        end
    end
    return exp_grid, transition_matrix
end

income_process, transition_matrix = generate_income_process(params)

([0.36787944117144233, 0.3682475046136629, 0.3686159363034188, 0.3689847366091417, 0.369353905899632, 0.3697234445440589, 0.3700933529119613, 0.37046363137324734, 0.37083428029819565, 0.37120530005745517  …  2.6939270528874992, 2.696622327353013, 2.6993202984410796, 2.7020209688496686, 2.7047243412794524, 2.7074304184338023, 2.7101392030187967, 2.7128506977432196, 2.715564905318567, 2.718281828459045], [0.0 1.0 … 0.0 0.0; 0.0004999999999999449 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0004999999999999449; 0.0 0.0 … 1.0 0.0])

In [23]:
function productivity_grid(params)
    (; z, p_star) = params
    grid = generate_income_process(params)
    p_grid = z .+ (grid[1] .* (p_star - z))
    return p_grid
end

p_grid  = productivity_grid(params)

2001-element Vector{Float64}:
 0.6207276647028654
 0.6209485027681978
 0.6211695617820513
 0.621390841965485
 0.6216123435397792
 0.6218340667264354
 0.6220560117471768
 0.6222781788239484
 0.6225005681789174
 0.6227231800344731
 ⋮
 2.017973396411808
 2.019592179064648
 2.021212581309801
 2.0228346047676715
 2.0244582510602815
 2.026083521811278
 2.0277104186459316
 2.02933894319114
 2.030969097075427

In [24]:
function separation_grid(params)
    (; n, s_star) = params
    grid = generate_income_process(params)
    s_grid = grid[1] .* s_star
    return s_grid
end

s_grid = separation_grid(params)

2001-element Vector{Float64}:
 0.036787944117144235
 0.03682475046136629
 0.03686159363034188
 0.03689847366091417
 0.0369353905899632
 0.03697234445440589
 0.037009335291196134
 0.037046363137324734
 0.037083428029819565
 0.03712053000574552
 ⋮
 0.2696622327353013
 0.26993202984410797
 0.2702020968849669
 0.27047243412794525
 0.2707430418433802
 0.2710139203018797
 0.271285069774322
 0.2715564905318567
 0.27182818284590454